In [2]:
import pandas as pd
df=pd.read_csv("car data.csv")
df

,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...,...
296,city,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,brio,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,city,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0


In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.sparse import csr_matrix
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("car data.csv")

X = data.drop(columns=['Selling_Price','Car_Name'])
y = data['Selling_Price']

categorical_cols = ['Fuel_Type','Selling_type','Transmission']
numeric_cols = [col for col in X.columns if col not in categorical_cols]

# ✅ Use sparse_output=True for new sklearn
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ]
)

model = Pipeline([
    ('preprocess', preprocessor),
    ('regressor', LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model.fit(X_train, y_train)

# Accuracy
from sklearn.metrics import r2_score
y_pred = model.predict(X_test)
print("Model Accuracy (R²):", round(r2_score(y_test, y_pred) * 100, 2), "%")

# Get the CSR matrix
X_transformed = model.named_steps['preprocess'].transform(X)
X_sparse = csr_matrix(X_transformed)
print("CSR Matrix shape:", X_sparse.shape)

# Example prediction
sample = pd.DataFrame([{
    'Year': 2016,
    'Present_Price': 8.5,
    'Driven_kms': 30000,
    'Fuel_Type': 'Petrol',
    'Selling_type': 'Dealer',
    'Transmission': 'Manual',
    'Owner': 0
}])
print("Predicted Selling Price:", round(model.predict(sample)[0], 2), "lakhs")


Model Accuracy (R²): 84.89 %
CSR Matrix shape: (301, 11)
Predicted Selling Price: 5.87 lakhs
